# 1.この課題の目的
- 自然言語処理を体験する
- 簡単な文書の分析ができるようになる
以下の要件をすべて満たしていた場合、合格とします。

※Jupyter Notebookを使い課題に沿った検証や説明ができている。

# 【問題1】BoWとN-gram(手計算)
目的

- 古典的かつ強力な手法BoWとN-gramの理解
以下は俳優K.Kさんのつぶやき(コーパス)です。
>文1: 今撮影中で〜す！
>
>文2: 今休憩中で〜す(^^)
>
>文3: 今日ドラマ撮影で〜す！
>    
>文4: 今日、映画瞬公開で〜す！！！

## 【問】
特殊文字除去(!や〜など)、単語分割をし以下の2パターンで文1〜文4を数値化(ベクトル化)してください。

- BoW(1-gram)
- BoW(2-gram)

手計算の後見やすい形にしてください。

例

In [105]:
import pandas as pd
import numpy as np

import re

from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSKeepFilter, CompoundNounFilter, TokenCountFilter

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from gensim.models.keyedvectors import KeyedVectors
import MeCab
import math
import pyprind
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
import os
from sklearn.datasets import load_files
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import linear_model
from sklearn.svm import SVC
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [24]:
vocabulary = ["I", "love", "this", "is", "the", "baseball"]
ms_kk_texts = ["I love baseball !!", "I love this !"]
texts_vec = [[1,1,0,0,0,1], [1,1,1,0,0,0]]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = ms_kk_texts)
df_bow_1gram

,I,love,this,is,the,baseball
I love baseball !!,1,1,0,0,0,1
I love this !,1,1,1,0,0,0


## BoW(1-gram)

In [2]:
vocabulary = ["今", "撮影", "中", "で〜す", "休憩", "今日","ドラマ", "映画", "瞬", "公開"]
ms_kk_texts = ["今撮影中で〜す！", "今休憩中で〜す(^^)", "今日ドラマ撮影で〜す！",\
               "今日、映画瞬公開で〜す！！！"]
texts_vec = [[1,1,1,1,0,0,0,0,0,0], [1,0,1,1,1,0,0,0,0,0],[0,1,0,1,0,1,1,0,0,0],\
             [0,0,0,1,0,1,0,1,1,1]]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = ms_kk_texts)
df_bow_1gram

,今,撮影,中,で〜す,休憩,今日,ドラマ,映画,瞬,公開
今撮影中で〜す！,1,1,1,1,0,0,0,0,0,0
今休憩中で〜す(^^),1,0,1,1,1,0,0,0,0,0
今日ドラマ撮影で〜す！,0,1,0,1,0,1,1,0,0,0
今日、映画瞬公開で〜す！！！,0,0,0,1,0,1,0,1,1,1


## BoW(2-gram)

In [3]:
vocabulary = ["今撮影", "撮影中", "中で〜す", "今休憩", "休憩中","中で〜す", "今日ドラマ",\
              "ドラマ撮影", "撮影で〜す", "今日映画", "映画瞬間","瞬公開", "公開で〜す"]


texts_vec = [[1,0,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,1,0,0,0,0,0,0,0],\
             [0,0,0,0,0,0,1,0,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,1,0,1,1]]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = ms_kk_texts)
df_bow_1gram

,今撮影,撮影中,中で〜す,今休憩,休憩中,中で〜す,今日ドラマ,ドラマ撮影,撮影で〜す,今日映画,映画瞬間,瞬公開,公開で〜す
今撮影中で〜す！,1,0,1,0,0,0,0,0,0,0,0,0,0
今休憩中で〜す(^^),0,0,0,1,0,1,0,0,0,0,0,0,0
今日ドラマ撮影で〜す！,0,0,0,0,0,0,1,0,1,0,0,0,0
今日、映画瞬公開で〜す！！！,0,0,0,0,0,0,0,0,0,1,0,1,1


# 【問題2】TF-IDF(手計算)
目的

- 古典的かつ強力なTF-IDFの理解

## 標準的なTF-IDFの公式 
$Term Frequency$:
$$
tf(t,d) = \frac{n_{t,d}}{\sum_{s \in d}n_{s,d}}
$$

$n_{t,d}$: 文書$d$内の単語$t$の出現回数

$\sum_{s \in d}n_{s,d}$
 : 文書$d$の全単語の出現回数の和


$Inverse Document Frequency$:

$$
idf(t) = \log_2{\frac{N}{df(t)}}
$$

$N$ : 全文書数

$df(t)$ : 単語$t$が出現する文書数

$TF-IDF$:
$$
tfidf(t, d) = tf(t, d) \times idf(t)
$$

## 【問】
問題1のコーパスを使って、文1〜文4をTFIDFで数値化(ベクトル化)してください。 
問題1と同様、手計算の後見やすい形にしてください。

### 正解例
tfidf(今, 文書1) = 0.25 になります。

In [4]:
## TFIDF(今、文書1)

In [5]:
text1 = ['今', '撮影', '中', 'で〜す']
text2 = ['今' ,'休憩', '中', 'で〜す']
text3 = ['今日' ,'ドラマ', '撮影', 'で〜す']
text4 = ['今日' ,'映画', '瞬', '公開', 'で〜す']

In [6]:
all_text = [text1, text2, text3, text4]
all_text

[['今', '撮影', '中', 'で〜す'],
 ['今', '休憩', '中', 'で〜す'],
 ['今日', 'ドラマ', '撮影', 'で〜す'],
 ['今日', '映画', '瞬', '公開', 'で〜す']]

## 例　TF-IDF(撮影, 文書1)

## TFの計算

In [20]:
def compute_tf(text, word):
    count = 0
    for t in text:
        if word == t:
            count += 1
    return count / len(text)

In [21]:
compute_tf(text1, '今')
print('TF={0:.2f}'.format(compute_tf(text1, '今')))

TF=0.25


## IDFの計算

In [25]:
def compute_idf(all_text, word):
    count = 0
    for t in all_text:
        if word in t:
            count += 1
    return np.log(len(all_text) / count)

In [26]:
print('IDF={0:.2f}'.format(compute_idf(all_text, '今')))

IDF=0.69


## TF-IDFの計算

In [29]:
def compute_tf_idf(text, all_text, word):
    return compute_tf(text, word) * compute_idf(all_text, word)

In [30]:
print('TF-IDF={0:.2f}'.format(compute_tf_idf(text1, all_text, '今')))

TF-IDF=0.17


## 例　TF-IDF(休憩, 文書2)

In [31]:
compute_tf_idf(text2, all_text, '休憩')

0.34657359027997264

In [32]:
print('TF-IDF={0:.2f}'.format(compute_tf_idf(text2, all_text, '休憩')))

TF-IDF=0.35


## 例　TF-IDF(ドラマ, 文書3)

In [33]:
compute_tf_idf(text3, all_text, 'ドラマ')

0.34657359027997264

In [34]:
print('TF-IDF={0:.2f}'.format(compute_tf_idf(text3, all_text, 'ドラマ')))

TF-IDF=0.35


## 例　TF-IDF(映画, 文書4)

In [35]:
compute_tf_idf(text4, all_text, '映画')

0.2772588722239781

In [36]:
print('TF-IDF={0:.2f}'.format(compute_tf_idf(text4, all_text, '映画')))

TF-IDF=0.28


# 【問題3】テキストクリーニング(プログラミング)
## 目的
- 実データ対応のためのテキストクリーニングの理解
- 正規表現の理解

実際のテキストデータは非常に汚いことが多いです。 
以下は3/6(水)にnoroさんがSlackで発言した文章で、良い例です。

＃<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:

# 【問】
このテキストに以下の処理を施してください。

- urlを削除
- 【〇〇】を削除
- 改行等の特殊文字を削除
- 絵文字除去

ここではしませんが、数字を文字列NUMBERに置き換える処理をよくします。

正解例

有償のRubyMineやPyCharmの6ヶ月間100%OFFクーポンをご希望者の方先着100名様に贈呈いたします！この度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。RubyMinePyCharm「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。期限は、2019年3月20日（水）22:00までとさせていただきます。ふるってのご希望をお待ちしております！

正規表現のサンプル

In [8]:
# 正規表現操作のライブラリ
import re
# 対象テキストデータ
text = '【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\n'
# re.compileを使うと処理が早くなります
BAD_SYMBOL = re.compile('[\n*！`]+')

# re.sub(r'[\n*！`]+', '', text)でもできます
text = re.sub(BAD_SYMBOL, '', text)
text

'【スペシャル特典】有償のRubyMineやPyCharmの 6ヶ月間100%OFFクーポン をご希望者の方先着100名様に贈呈いたします'

リアルタイムで正規表現を確認できるサイトです。

[https://regex101.com/](https://regex101.com/)

[https://regex-testdrive.com/ja/dotest](https://regex-testdrive.com/ja/)

[re 正規表現操作](https://docs.python.org/ja/3/library/re.html)

## Tips NLPのLinuxコマンド
これまでpythonでファイルを読み込んで処理をしていましたが、
簡単な作業においてはlinuxコマンドの方がメモリの使用料が半分以下だったりとパフォーマンスが良いです。

例えばファイルの行数を数えたい場合、pythonでわざわざ書くのは面倒です。
以下の1行のコマンドで実行できます。

>wc -l 〇〇.txt

また分割したい場合はsplit

並び替えたい場合はsort

置換にはsed

文の先頭、後頭部分を見たければhead,tail

など便利なコマンドがあります。

詳しく知りたい方はNLP100本ノックで調べてみてください。

In [37]:
text = '<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！\
*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\
\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n\
「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\
\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:'

symbol_reg = r'[\n*` ]+'
mention_reg = r'<.*?>'
phraze_reg = r'【.*?】'
command_reg = r':[^0-9０-９]+:'

reg_str = f'{symbol_reg}|{mention_reg}|{phraze_reg}|{command_reg}'
reg_str = re.compile(reg_str)

# re.sub(r'[\n*！`]+', '', text)でもできます
text = re.sub(reg_str, '', text)
text

'有償のRubyMineやPyCharmの6ヶ月間100%OFFクーポンをご希望者の方先着100名様に贈呈いたします！この度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。RubyMinePyCharm「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。期限は、2019年3月20日（水）22:00までとさせていただきます。ふるってのご希望をお待ちしております！'

# 【問題4】形態素解析
目的

- 形態素解析の理解
形態素解析のツールはMecabやJanomeなど様々ですが、
ここでは手軽に導入できるJanomeを使います。

[Janome document](https://mocobeta.github.io/janome/)

【問】
上記のクリーニングしたテキストをJanomeを用いて形態素解析をし、
名詞または動詞の単語を抜き出してください。

正解例
>["有償", "RubyMine", "Pycharm", ...]

In [43]:
t = Tokenizer()
for token in t.tokenize(text):
    print(token)

有償	名詞,一般,*,*,*,*,有償,ユウショウ,ユーショー
の	助詞,連体化,*,*,*,*,の,ノ,ノ
RubyMine	名詞,一般,*,*,*,*,RubyMine,*,*
や	助詞,並立助詞,*,*,*,*,や,ヤ,ヤ
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
の	助詞,連体化,*,*,*,*,の,ノ,ノ
6	名詞,数,*,*,*,*,6,*,*
ヶ月	名詞,接尾,助数詞,*,*,*,ヶ月,カゲツ,カゲツ
間	名詞,接尾,一般,*,*,*,間,カン,カン
100	名詞,数,*,*,*,*,100,*,*
%	名詞,サ変接続,*,*,*,*,%,*,*
OFF	名詞,一般,*,*,*,*,OFF,*,*
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
ご	接頭詞,名詞接続,*,*,*,*,ご,ゴ,ゴ
希望	名詞,サ変接続,*,*,*,*,希望,キボウ,キボー
者	名詞,接尾,一般,*,*,*,者,シャ,シャ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
方	名詞,非自立,一般,*,*,*,方,ホウ,ホー
先着	名詞,サ変接続,*,*,*,*,先着,センチャク,センチャク
100	名詞,数,*,*,*,*,100,*,*
名	名詞,接尾,助数詞,*,*,*,名,メイ,メイ
様	名詞,接尾,人名,*,*,*,様,サマ,サマ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
贈呈	名詞,サ変接続,*,*,*,*,贈呈,ゾウテイ,ゾーテイ
いたし	動詞,非自立,*,*,五段・サ行,連用形,いたす,イタシ,イタシ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
！	記号,一般,*,*,*,*,！,！,！
この	連体詞,*,*,*,*,*,この,コノ,コノ
度	名詞,非自立,副詞可能,*,*,*,度,タビ,タビ
、	記号,読点,*,*,*,*,、,、,、
RubyMine	名詞,固有名詞,組織,*,*,*,RubyMine,*,*
や	助詞,並立助詞,*,*,*,*,や,ヤ,ヤ
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
の	助詞,連体化,*,*,*,*,の,ノ,ノ
メーカー	名詞,一般,*,*,*,*,メーカー,メーカー,メーカー
で	助

In [49]:
for token in t.tokenize(text):
    if token.part_of_speech.startswith('名詞,一般'):
        print(token.surface)

有償
RubyMine
PyCharm
OFF
クーポン
PyCharm
メーカー
JetBrains
クーポン
コード
クーポン
RubyMinePyCharm
手
方式
手
スタンプ
クリック
期限
月
水


# 【問題5】ニュースの分析
目的

- 日本語の自然言語処理の体験
- 類似度の理解
以下からldcc-20140209.tar.gzをダウンロードしてください。 

[livedoor](https://www.rondhuit.com/download.html#ldcc)

もしくはwgetコマンドを使っても良いです。



In [15]:
# # livedoorのnewsをダウンロード
# wget 'https://www.rondhuit.com/download/ldcc-20140209.tar.gz'
# # 圧縮ファイルを解凍
# tar zxf ldcc-20140209.tar.gz
# # livedoorニュースの説明を表示
# cat text/README.txt

サンプルコード

In [57]:
# サブフォルダまで賢く読み込んでもらう
from sklearn.datasets import load_files
# encodingをutf-8指定して読み込み
bin_data = load_files('./text', encoding='utf-8')
documents = bin_data.data
# 今回はラベルが無いと仮定してください
# targets = bin_data.target

【問】
以下の流れでニュースを分析してください。

- まずどんなニュースなのか読んでみる
- 出現単語をカウントして分析する
- テキストをクリーニングする
- BoW + TFIDFでベクトル化する
- あるニュースに一番cos類似度が近いニュースを出力する関数の作成
- 別の類似度手法を1つ調べて上の関数に組み込む(切り替えられるようにする)
- なぜそのような結果になったのか考察する

[sklearn.feature_extraction.text](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

## どんなニュースなのか？
### 様々なジャンル(食事、芸能、IT、ゲーム)の出来事や話題となっている物などについて記載されている

In [62]:
len(documents)

7376

In [58]:
documents[0]

'http://news.livedoor.com/article/detail/4931238/\n2010-08-08T10:00:00+0900\nNY名物イベントが日本でも！名店グルメを気軽に楽しむ\nニューヨークで20年続いている食の祭典「レストラン・ウィーク」。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山・六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円（税・サ別）、ディナー5000円（税・サ別）を気軽に楽しめる、とっておきのイベントです。\n\u3000\n\u3000実行委員長には、学校法人服部学園、服部栄養専門学校 理事長・校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LA BETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家・脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。\n\n参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフの「レストラン タテル ヨシノ 汐留」や、日本料理の名門「つきぢ田村」、「金田中 庵」、「赤坂璃宮」に「mikuni MARUNOUCHI」など、日本を代表するレストランがずらり。\n\u3000イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。\n\n\u3000予約方法は必ず事前に、各店舗に問合せを行い「ジャパンレストラン・ウィーク2010」での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できる「ジャパンレストラン・ウィーク2010」にぜひ参加しててみてはいかがですか？\n\nJAPAN RESTAURANT WEEK 2010 -公式サイト\n'

In [59]:
documents[1]

'http://news.livedoor.com/article/detail/6655079/\n2012-06-13T19:25:00+0900\n小沢一郎氏の妻が支援者に離婚を報告。「週刊文春」報じる\n13日、Web版「週刊文春」は、民主党の元代表・小沢一郎氏の妻が、支援者宛に離婚したことを伝える手紙を送ったと報じ、ツイッターやネット掲示板で大きな話題になっている。\n\n記事によると、その手紙は「小沢は放射能が怖くて秘書と一緒に逃げだしました」「隠し子が発覚した際、小沢元代表は和子夫人に謝るどころか、『いつでも離婚してやる』と言い放ち、和子夫人は一時は自殺まで考えた」という小沢氏の支持者にとってはショッキングな内容となっている。\n\nツイッターやネット掲示板では、「これが小沢一郎の本性か」「こりゃすごいな。てか、がっかりだなあ」などと、手紙の内容に驚く声が相次ぎ、その他にも「小沢潰しですかね?」「元秘書が交番で暴れて逮捕された件といい、小沢氏を潰したい何者かによる工作活動では?」といった声が挙がっていた。\n\n【関連情報】\n・小沢一郎夫人が支援者に「離婚しました」（週刊文春）\n'

In [60]:
documents[2]

'http://news.livedoor.com/article/detail/5172424/\n2010-11-30T08:00:00+0900\n【Sports Watch】田中＆里田の交際、アプローチは里田から\nグラビアアイドル・ほしのあき＆騎手・三浦皇成がお互いのブログで交際を認めた日、東北楽天ゴールデンイーグルスのエース・田中将大とタレント・里田まいの交際もまた公のものとなったが、本日30日（火）発売の「週刊アサヒ芸能」（12.9号）では、「マー君を“ナンパ”した里田」との見出しで、両者の交際にまつわる関係者の証言を紹介した。\n\n同誌にコメントを寄せた芸能デスクによると、「周囲の事情はさておき、当人たちが盛り上がっているのは確か。結婚も完全に視野に入れているようで、すでに新居を探しているとの話まである」という。\n\nまた、二人の交際は里田からのアプローチによるものとのことで、前出の芸能デスクは、「2人は昨年末の番組共演時に、里田から猛アタック。『一緒に食事でも』と誘い、春頃にはつきあい始めた。里田は仲のいいスザンヌ（24）、木下優樹菜（22）には、早くから相談していたし、周囲にも浮かれてしゃべりまくっていた。次世代エースをゲットしたわけですから、賞味期限切れ間近の里田にすれば、してやったりでしょう」とも語っている。\n\nちなみに、同じく同誌にコメントする球団関係者は、「高校時代は同じ高校の女子生徒と在学中の3年間、ずっとつきあっていましたね。基本はオクテ。プロ入り後にキャバクラなども覚えましたが、まぁ、モテるタイプではないし、女あしらいも不慣れ。高校時代はかなりワガママで、元カノは苦労が絶えなかったと聞いています。同年代の女より、里田のような姉さんタイプのほうが勝負の世界では向いているケースも多いですからね」と明かしており、この交際も田中にとっては“吉”とした。\n\n・週刊アサヒ芸能 ［ライト版］＜デジタル＞（PC版）\n・週刊アサヒ芸能（モバイル版）\n'

In [61]:
documents[-1]

'http://news.livedoor.com/article/detail/5039076/\n2010-09-29T01:00:00+0900\n予約締切は本日まで！「キン肉マン」10周年Ｔシャツイベント\n人気漫画「キン肉マン」のアパレルシリーズが今年で10年目となる記念イヤーを迎えた。\n\n本シリーズをプロデュースするBAMBAMBIGELOW＆livedoorのコラボーレーション企画として、先月末より、キン肉マンに登場する超人10名のオリジナルＴシャツを期間限定で発売しており、いよいよ今日29日に、その予約注文締切日を迎える。\n\n10月29日（金）、日本記念日協会が定める「キン肉マンの日」（29日の金曜日）に注文したＴシャツが届く同イベント——、購入者の中から抽選で10名にプレゼントが当たる特典も付いている同企画だが、そのプレゼントは、著者のサイン入りソフビや、バッファローマンの版画といったファン必見の内容になっている。\n\n泣いても笑ってもラスト一日。特設サイトを是非チェックしてほしい。\n\n・「キン肉マン」“マッスルアパレル”コラボ10周年記念 特設サイト\n'

## documetsの文章が膨大な為、以下の構文解析は個人的に面白い文章と感じたdocuments[2]のみについて行う
## 出現単語をカウントして分析する

In [69]:
token_filters = [POSKeepFilter('名詞, 一般'), TokenCountFilter()]
a = Analyzer(token_filters=token_filters)
top_words = list(a.analyze(documents[2]))
top_words

[('http', 1),
 ('://', 1),
 ('news', 1),
 ('.', 3),
 ('livedoor', 1),
 ('com', 1),
 ('/', 4),
 ('article', 1),
 ('detail', 1),
 ('5172424', 1),
 ('2010', 1),
 ('-', 2),
 ('11', 1),
 ('30', 2),
 ('T', 1),
 ('08', 1),
 (':', 2),
 ('00', 2),
 ('+', 1),
 ('0900', 1),
 ('Sports', 1),
 ('Watch', 1),
 ('田中', 3),
 ('里', 9),
 ('田', 9),
 ('交際', 6),
 ('アプローチ', 2),
 ('グラビア', 1),
 ('アイドル', 1),
 ('あき', 1),
 ('騎手', 1),
 ('三浦', 1),
 ('皇', 1),
 ('成', 1),
 ('お互い', 1),
 ('ブログ', 1),
 ('日', 2),
 ('東北', 1),
 ('楽天', 1),
 ('ゴールデンイーグルス', 1),
 ('エース', 2),
 ('将', 1),
 ('大', 1),
 ('タレント', 1),
 ('公', 1),
 ('もの', 2),
 ('本日', 1),
 ('火', 1),
 ('発売', 1),
 ('週刊', 3),
 ('アサヒ', 3),
 ('芸能', 5),
 ('12', 1),
 ('9', 1),
 ('号', 1),
 ('マー', 1),
 ('君', 1),
 ('ナンパ', 1),
 ('見出し', 1),
 ('両者', 1),
 ('関係', 2),
 ('者', 2),
 ('証言', 1),
 ('紹介', 1),
 ('同誌', 2),
 ('コメント', 2),
 ('デスク', 2),
 ('周囲', 2),
 ('事情', 1),
 ('当人', 1),
 ('たち', 1),
 ('の', 1),
 ('確か', 1),
 ('結婚', 1),
 ('完全', 1),
 ('視野', 1),
 ('よう', 2),
 ('新居', 1),
 ('話', 1),
 ('二', 1),

## テキストをクリーニングする

In [74]:
reg1 = '<.*?>'
reg2 = '【.*?】'
reg3 = '[\n\*`\s■◆○★●]'
reg4 = '(http.*?\n)'
reg5 = '(\d{4}-.*?\+\d{4})'

reg_str = f'{kakko1}|{kakko2}|{tokusyu}|{url}|{date}'

In [75]:
document = []

for i in documents:
    remove = re.compile(reg_str)
    document.append(re.sub(remove, '', i))

print(document[2])

田中＆里田の交際、アプローチは里田からグラビアアイドル・ほしのあき＆騎手・三浦皇成がお互いのブログで交際を認めた日、東北楽天ゴールデンイーグルスのエース・田中将大とタレント・里田まいの交際もまた公のものとなったが、本日30日（火）発売の「週刊アサヒ芸能」（12.9号）では、「マー君を“ナンパ”した里田」との見出しで、両者の交際にまつわる関係者の証言を紹介した。同誌にコメントを寄せた芸能デスクによると、「周囲の事情はさておき、当人たちが盛り上がっているのは確か。結婚も完全に視野に入れているようで、すでに新居を探しているとの話まである」という。また、二人の交際は里田からのアプローチによるものとのことで、前出の芸能デスクは、「2人は昨年末の番組共演時に、里田から猛アタック。『一緒に食事でも』と誘い、春頃にはつきあい始めた。里田は仲のいいスザンヌ（24）、木下優樹菜（22）には、早くから相談していたし、周囲にも浮かれてしゃべりまくっていた。次世代エースをゲットしたわけですから、賞味期限切れ間近の里田にすれば、してやったりでしょう」とも語っている。ちなみに、同じく同誌にコメントする球団関係者は、「高校時代は同じ高校の女子生徒と在学中の3年間、ずっとつきあっていましたね。基本はオクテ。プロ入り後にキャバクラなども覚えましたが、まぁ、モテるタイプではないし、女あしらいも不慣れ。高校時代はかなりワガママで、元カノは苦労が絶えなかったと聞いています。同年代の女より、里田のような姉さんタイプのほうが勝負の世界では向いているケースも多いですからね」と明かしており、この交際も田中にとっては“吉”とした。・週刊アサヒ芸能［ライト版］＜デジタル＞（PC版）・週刊アサヒ芸能（モバイル版）


# BoW + TFIDFでベクトル化する

### 文章から名詞を抽出する関数

In [173]:
def split_text_only_noun(text):
    tagger = MeCab.Tagger()
    words = []
    for c in tagger.parse(text).splitlines()[:-1]:
        surface, feature = c.split('\t')
        pos = feature.split(',')[0]
        if pos == '名詞':
            words.append(surface)
    return ' '.join(words)

### 名詞を抽出しリスト化する

In [182]:
message_list = [split_text_only_noun(documents[2])]
message_list

['http :// news . livedoor . com / article / detail / 5172424 / 2010 - 11 - 30 T 08 : 00 : 00 + 0900 Sports Watch 田中 里 田 交際 アプローチ 里 田 グラビア アイドル あき 騎手 三浦 皇 成 お互い ブログ 交際 日 東北 楽天 ゴールデンイーグルス エース 田中 将 大 タレント 里 田 交際 公 もの 本日 30 日 火 発売 週刊 アサヒ 芸能 12 . 9 号 マー 君 ナンパ 里 田 見出し 両者 交際 関係 者 証言 紹介 同誌 コメント 芸能 デスク 周囲 事情 当人 たち の 確か 結婚 完全 視野 よう 新居 話 二 人 交際 里 田 アプローチ もの こと 出 芸能 デスク 2 人 昨年 末 番組 共演 時 里 田 アタック 一緒 食事 春 頃 里 田 仲 スザンヌ 24 木下 優樹 菜 22 相談 周囲 次世代 エース ゲット わけ 賞味 期限切れ 間近 里 田 同誌 コメント 球団 関係 者 高校 時代 高校 女子 生徒 在学 中 3 年間 基本 オクテ プロ 入り 後 キャバクラ モテ タイプ 女 あしらい 不慣れ 高校 時代 ワガママ カノ 苦労 同年代 女 里 田 よう 姉さん タイプ ほう 勝負 世界 ケース 交際 田中 吉 週刊 アサヒ 芸能 ライト 版 デジタル PC 版 週刊 アサヒ 芸能 モバイル 版']

### numpy配列に変換

In [183]:
docs = np.array(message_list)
docs

array(['http :// news . livedoor . com / article / detail / 5172424 / 2010 - 11 - 30 T 08 : 00 : 00 + 0900 Sports Watch 田中 里 田 交際 アプローチ 里 田 グラビア アイドル あき 騎手 三浦 皇 成 お互い ブログ 交際 日 東北 楽天 ゴールデンイーグルス エース 田中 将 大 タレント 里 田 交際 公 もの 本日 30 日 火 発売 週刊 アサヒ 芸能 12 . 9 号 マー 君 ナンパ 里 田 見出し 両者 交際 関係 者 証言 紹介 同誌 コメント 芸能 デスク 周囲 事情 当人 たち の 確か 結婚 完全 視野 よう 新居 話 二 人 交際 里 田 アプローチ もの こと 出 芸能 デスク 2 人 昨年 末 番組 共演 時 里 田 アタック 一緒 食事 春 頃 里 田 仲 スザンヌ 24 木下 優樹 菜 22 相談 周囲 次世代 エース ゲット わけ 賞味 期限切れ 間近 里 田 同誌 コメント 球団 関係 者 高校 時代 高校 女子 生徒 在学 中 3 年間 基本 オクテ プロ 入り 後 キャバクラ モテ タイプ 女 あしらい 不慣れ 高校 時代 ワガママ カノ 苦労 同年代 女 里 田 よう 姉さん タイプ ほう 勝負 世界 ケース 交際 田中 吉 週刊 アサヒ 芸能 ライト 版 デジタル PC 版 週刊 アサヒ 芸能 モバイル 版'],
      dtype='<U638')

### BoWの実施

In [184]:
count = CountVectorizer()
bags = count.fit_transform(docs)
features = count.get_feature_names()

In [187]:
print('bag : ', bags)
print(bags.toarray())
print("count.vocabulary_ : ", count.vocabulary_)
print('features:', features)
print(features)
print("---------------------------------------------------")

bag :    (0, 51)	1
  (0, 16)	1
  (0, 44)	1
  (0, 52)	1
  (0, 37)	1
  (0, 57)	1
  (0, 64)	1
  (0, 24)	1
  (0, 71)	1
  (0, 65)	1
  (0, 94)	1
  (0, 34)	1
  (0, 53)	1
  (0, 56)	1
  (0, 20)	1
  (0, 42)	2
  (0, 50)	1
  (0, 35)	1
  (0, 62)	1
  (0, 48)	1
  (0, 33)	1
  (0, 69)	1
  (0, 73)	1
  (0, 68)	1
  (0, 85)	1
  :	:
  (0, 47)	1
  (0, 21)	1
  (0, 55)	1
  (0, 103)	1
  (0, 19)	1
  (0, 28)	1
  (0, 36)	1
  (0, 31)	2
  (0, 60)	6
  (0, 86)	3
  (0, 18)	1
  (0, 17)	1
  (0, 2)	1
  (0, 0)	2
  (0, 1)	1
  (0, 8)	2
  (0, 3)	1
  (0, 5)	1
  (0, 9)	1
  (0, 12)	1
  (0, 10)	1
  (0, 11)	1
  (0, 14)	1
  (0, 15)	1
  (0, 13)	1
[[2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 3 1 2 2 1 1 1
  1 1 1 2 1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 6 1 1 1 1 1 2 2 1 1 1 1
  1 1 1 1 1 2 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 5 1 1 1 1 1 3 1 2 1 1 3]]
count.vocabulary_ :  {'http': 13, 'news': 15, 'livedoor': 14, 'com': 11, 'article': 10, 'detail': 12, '5172424': 9, '2010': 5, '11': 3, '30': 8, '08': 1, '00': 0, '0900': 2, '

### BoWの結果からTFIDFを計算

In [190]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
tf_idf = tfidf.fit_transform(bags)
print(tf_idf)
print(tf_idf.toarray())

  (0, 104)	0.19569842191603265
  (0, 103)	0.06523280730534423
  (0, 102)	0.06523280730534423
  (0, 101)	0.13046561461068845
  (0, 100)	0.06523280730534423
  (0, 99)	0.19569842191603265
  (0, 98)	0.06523280730534423
  (0, 97)	0.06523280730534423
  (0, 96)	0.06523280730534423
  (0, 95)	0.06523280730534423
  (0, 94)	0.06523280730534423
  (0, 93)	0.3261640365267211
  (0, 92)	0.06523280730534423
  (0, 91)	0.06523280730534423
  (0, 90)	0.06523280730534423
  (0, 89)	0.06523280730534423
  (0, 88)	0.06523280730534423
  (0, 87)	0.06523280730534423
  (0, 86)	0.19569842191603265
  (0, 85)	0.06523280730534423
  (0, 84)	0.06523280730534423
  (0, 83)	0.06523280730534423
  (0, 82)	0.06523280730534423
  (0, 81)	0.06523280730534423
  (0, 80)	0.06523280730534423
  :	:
  (0, 24)	0.06523280730534423
  (0, 23)	0.06523280730534423
  (0, 22)	0.06523280730534423
  (0, 21)	0.06523280730534423
  (0, 20)	0.06523280730534423
  (0, 19)	0.06523280730534423
  (0, 18)	0.06523280730534423
  (0, 17)	0.06523280730534423


## あるニュースに一番cos類似度が近いニュースを出力する関数の作成

## documentsの中から指定した文章に一番cos類似度が高い文章を出力する関数

In [ ]:
def output_news_with_cos_similarity(source_id):
    
    # 文字列を単語で分割しリストに格納する
    def word_tokenaize(doc):
        tagger = MeCab.Tagger('-Owakati')
        # 初期化しないとエラーになる
        tagger.parse("")
        
        node = tagger.parseToNode(doc)

        result = []
        while node:
            hinshi = node.feature.split(",")[0]
            if hinshi == '名詞' or hinshi == '動詞':
                result.append(node.feature.split(",")[6])
            node = node.next

        return result
    
    
    def words_to_freqdict(words):
        """
        単語の配列を、単語と頻度の辞書に変換する関数
        例: ["X","X","Y","Z","X"] => {"X":3, "Y":1, "Z":1}
        @param words 単語の配列
        @return 単語と頻度の辞書
        """
        freqdict = {}
        for word in words:
            if word in freqdict:
                freqdict[word] = freqdict[word] + 1
            else:
                freqdict[word] = 1
        return freqdict
    
    
    def calc_cos(dictA, dictB):
        """
        cos類似度を計算する関数
        @param dictA 1つ目の文章
        @param dictB 2つ目の文章
        @return cos類似度を計算した結果。0〜1で1に近ければ類似度が高い。
        """
        # 文書Aのベクトル長を計算
        lengthA = 0.0
        for key,value in dictA.items():
            lengthA = lengthA + value*value
        lengthA = math.sqrt(lengthA)

        # 文書Bのベクトル長を計算
        lengthB = 0.0
        for key,value in dictB.items():
            lengthB = lengthB + value*value
        lengthB = math.sqrt(lengthB)

        # AとBの内積を計算
        dotProduct = 0.0
        for keyA,valueA in dictA.items():
            for keyB,valueB in dictB.items():
                if keyA==keyB:
                    dotProduct = dotProduct + valueA*valueB
        # cos類似度を計算
        cos = dotProduct / (lengthA*lengthB)
        return cos
    
    
    def compute_cos_sim(s_id, d_id):
        source_list = word_tokenaize(documents[s_id])
        destination_list = word_tokenaize(documents[d_id])
        
        source_freq_list = words_to_freqdict(source_list)
        destination_freq_list = words_to_freqdict(destination_list)
        
        cos_sim = calc_cos(source_freq_list, destination_freq_list)
        
        return cos_sim
    
    def main():
        documents = bin_data.data
        document_num = len(documents)

        print('比較元の文章 documents[{}]'.format(source_id))
        print('{}'.format(documents[int(source_id)]))

        similer_document_id = 0
        cos_sim = compute_cos_sim(int(source_id), 0)

        for index in range(document_num):
            test_cos_sim = compute_cos_sim(int(source_id), int(index))

            if source_id != index:
                if cos_sim < test_cos_sim:
                    cos_sim = test_cos_sim
                    similer_document_id = index

        print('最もcos類似度が高い文章　documents[{}]'.format(similer_document_id))
        print('cos類似度={}'.format(cos_sim))
        print('{}'.format(documents[int(similer_document_id)]))

    
    if __name__ == "__main__":
        main()

### 動作検証

In [172]:
output_news_with_cos_similarity(2)

比較元の文章 documents[2]
http://news.livedoor.com/article/detail/5172424/
2010-11-30T08:00:00+0900
【Sports Watch】田中＆里田の交際、アプローチは里田から
グラビアアイドル・ほしのあき＆騎手・三浦皇成がお互いのブログで交際を認めた日、東北楽天ゴールデンイーグルスのエース・田中将大とタレント・里田まいの交際もまた公のものとなったが、本日30日（火）発売の「週刊アサヒ芸能」（12.9号）では、「マー君を“ナンパ”した里田」との見出しで、両者の交際にまつわる関係者の証言を紹介した。

同誌にコメントを寄せた芸能デスクによると、「周囲の事情はさておき、当人たちが盛り上がっているのは確か。結婚も完全に視野に入れているようで、すでに新居を探しているとの話まである」という。

また、二人の交際は里田からのアプローチによるものとのことで、前出の芸能デスクは、「2人は昨年末の番組共演時に、里田から猛アタック。『一緒に食事でも』と誘い、春頃にはつきあい始めた。里田は仲のいいスザンヌ（24）、木下優樹菜（22）には、早くから相談していたし、周囲にも浮かれてしゃべりまくっていた。次世代エースをゲットしたわけですから、賞味期限切れ間近の里田にすれば、してやったりでしょう」とも語っている。

ちなみに、同じく同誌にコメントする球団関係者は、「高校時代は同じ高校の女子生徒と在学中の3年間、ずっとつきあっていましたね。基本はオクテ。プロ入り後にキャバクラなども覚えましたが、まぁ、モテるタイプではないし、女あしらいも不慣れ。高校時代はかなりワガママで、元カノは苦労が絶えなかったと聞いています。同年代の女より、里田のような姉さんタイプのほうが勝負の世界では向いているケースも多いですからね」と明かしており、この交際も田中にとっては“吉”とした。

・週刊アサヒ芸能 ［ライト版］＜デジタル＞（PC版）
・週刊アサヒ芸能（モバイル版）

最もcos類似度が高い文章　documents[7190]
cos類似度=0.9398907269721967
http://news.livedoor.com/article/detail/5377551/
2011-03-01T08:00:00+0900
【Sports 

## 別の類似度手法を1つ調べて上の関数に組み込む(切り替えられるようにする)

### ◯Word Mover's Distanceを導入
#### Word Mover's Distanceとは
- Word2vec や GloVe 等で得られた単語の分散表現を使って文書間の距離を計算する手法
- 値が0に近いほど似た文章であるということを示している　※cos類似度は1に近いほど似た文章となる
- Twitterのようなショートテキストに対して有効という研究結果が多い為、今回のような長い文章では対応できない可能性あり

#### 導入にあたっての準備
- 使用する分散表現は以下のページから学習済みのデータ(GoogleNews-vectors-negative300.bin.gz)をダウンロードし使用する
- 上記のデータは単語ベクトルは300次元で、ボキャブラリ数が300万、学習にはGoogle Newsコーパス(1000億語)を使用している。

[https://code.google.com/archive/p/word2vec/](https://code.google.com/archive/p/word2vec/)

### 動作例「japanese」にWMDが近い単語を表示

In [195]:
model = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

In [194]:
model.most_similar(positive=['japanese'])

[('japan', 0.6607722640037537),
 ('chinese', 0.6502295732498169),
 ('Japanese', 0.6149078607559204),
 ('korean', 0.6051568984985352),
 ('german', 0.5999273061752319),
 ('american', 0.5906797647476196),
 ('asian', 0.5839767456054688),
 ('san', 0.5834757089614868),
 ('jap', 0.5764404535293579),
 ('swedish', 0.5720360279083252)]

## 関数にWord Mover's Distanceを組み込む

In [200]:
def output_news_with_cos_similarity(source_id, method):
    
    # 文字列を単語で分割しリストに格納する
    def word_tokenaize(doc):
        tagger = MeCab.Tagger('-Owakati')
        # 初期化しないとエラーになる
        tagger.parse("")
        
        node = tagger.parseToNode(doc)

        result = []
        while node:
            hinshi = node.feature.split(",")[0]
            if hinshi == '名詞' or hinshi == '動詞':
                result.append(node.feature.split(",")[6])
            node = node.next

        return result
    
    
    def words_to_freqdict(words):
        """
        単語の配列を、単語と頻度の辞書に変換する関数
        例: ["X","X","Y","Z","X"] => {"X":3, "Y":1, "Z":1}
        @param words 単語の配列
        @return 単語と頻度の辞書
        """
        freqdict = {}
        for word in words:
            if word in freqdict:
                freqdict[word] = freqdict[word] + 1
            else:
                freqdict[word] = 1
        return freqdict
    
    
    def calc_cos(dictA, dictB):
        """
        cos類似度を計算する関数
        @param dictA 1つ目の文章
        @param dictB 2つ目の文章
        @return cos類似度を計算した結果。0〜1で1に近ければ類似度が高い。
        """
        # 文書Aのベクトル長を計算
        lengthA = 0.0
        for key,value in dictA.items():
            lengthA = lengthA + value*value
        lengthA = math.sqrt(lengthA)

        # 文書Bのベクトル長を計算
        lengthB = 0.0
        for key,value in dictB.items():
            lengthB = lengthB + value*value
        lengthB = math.sqrt(lengthB)

        # AとBの内積を計算
        dotProduct = 0.0
        for keyA,valueA in dictA.items():
            for keyB,valueB in dictB.items():
                if keyA==keyB:
                    dotProduct = dotProduct + valueA*valueB
        # cos類似度を計算
        cos = dotProduct / (lengthA*lengthB)
        return cos
    
    
    def compute_cos_sim(s_id, d_id):
        source_list = word_tokenaize(documents[s_id])
        destination_list = word_tokenaize(documents[d_id])
        
        source_freq_list = words_to_freqdict(source_list)
        destination_freq_list = words_to_freqdict(destination_list)
        
        cos_sim = calc_cos(source_freq_list, destination_freq_list)
        
        return cos_sim
    

    
    def main():
        documents = bin_data.data
        document_num = len(documents)
        print('使用する類似度手法 {}'.format(method))
        print('比較元の文章 documents[{}]'.format(source_id))
        print('{}'.format(documents[int(source_id)]))

        similer_document_id = 0
        
        if method == 'cos':
            sim = compute_cos_sim(int(source_id), 0)

            for index in range(document_num):
                test_cos_sim = compute_cos_sim(int(source_id), int(index))

                if source_id != index:
                    if sim < test_cos_sim:
                        sim = test_cos_sim
                        similer_document_id = index
        
        if method == 'wmd':
            sim = model.wmdistance(documents[int(source_id)], documents[0])
            
            for index in range(document_num):
                test_wmd_sim = model.wmdistance(documents[int(source_id)], documents[int(index)])

                if source_id != index:
                    if sim > test_wmd_sim:
                        sim = test_wmd_sim
                        similer_document_id = index
            

        print('最も類似度が高い文章　documents[{}]'.format(similer_document_id))
        print('類似度={}'.format(sim))
        print('{}'.format(documents[int(similer_document_id)]))

    
    if __name__ == "__main__":
        main()

### Cos類似度を使用した場合

In [213]:
output_news_with_cos_similarity(2, 'cos')

使用する類似度手法 cos
比較元の文章 documents[2]
http://news.livedoor.com/article/detail/5172424/
2010-11-30T08:00:00+0900
【Sports Watch】田中＆里田の交際、アプローチは里田から
グラビアアイドル・ほしのあき＆騎手・三浦皇成がお互いのブログで交際を認めた日、東北楽天ゴールデンイーグルスのエース・田中将大とタレント・里田まいの交際もまた公のものとなったが、本日30日（火）発売の「週刊アサヒ芸能」（12.9号）では、「マー君を“ナンパ”した里田」との見出しで、両者の交際にまつわる関係者の証言を紹介した。

同誌にコメントを寄せた芸能デスクによると、「周囲の事情はさておき、当人たちが盛り上がっているのは確か。結婚も完全に視野に入れているようで、すでに新居を探しているとの話まである」という。

また、二人の交際は里田からのアプローチによるものとのことで、前出の芸能デスクは、「2人は昨年末の番組共演時に、里田から猛アタック。『一緒に食事でも』と誘い、春頃にはつきあい始めた。里田は仲のいいスザンヌ（24）、木下優樹菜（22）には、早くから相談していたし、周囲にも浮かれてしゃべりまくっていた。次世代エースをゲットしたわけですから、賞味期限切れ間近の里田にすれば、してやったりでしょう」とも語っている。

ちなみに、同じく同誌にコメントする球団関係者は、「高校時代は同じ高校の女子生徒と在学中の3年間、ずっとつきあっていましたね。基本はオクテ。プロ入り後にキャバクラなども覚えましたが、まぁ、モテるタイプではないし、女あしらいも不慣れ。高校時代はかなりワガママで、元カノは苦労が絶えなかったと聞いています。同年代の女より、里田のような姉さんタイプのほうが勝負の世界では向いているケースも多いですからね」と明かしており、この交際も田中にとっては“吉”とした。

・週刊アサヒ芸能 ［ライト版］＜デジタル＞（PC版）
・週刊アサヒ芸能（モバイル版）

最もcos類似度が高い文章　documents[7190]
類似度=0.9398907269721967
http://news.livedoor.com/article/detail/5377551/
2011-03-01T08:00:00+09

### Word Mover's Distanceを使用した場合

In [201]:
output_news_with_cos_similarity(2, 'wmd')

使用する類似度手法 wmd
比較元の文章 documents[2]
http://news.livedoor.com/article/detail/5172424/
2010-11-30T08:00:00+0900
【Sports Watch】田中＆里田の交際、アプローチは里田から
グラビアアイドル・ほしのあき＆騎手・三浦皇成がお互いのブログで交際を認めた日、東北楽天ゴールデンイーグルスのエース・田中将大とタレント・里田まいの交際もまた公のものとなったが、本日30日（火）発売の「週刊アサヒ芸能」（12.9号）では、「マー君を“ナンパ”した里田」との見出しで、両者の交際にまつわる関係者の証言を紹介した。

同誌にコメントを寄せた芸能デスクによると、「周囲の事情はさておき、当人たちが盛り上がっているのは確か。結婚も完全に視野に入れているようで、すでに新居を探しているとの話まである」という。

また、二人の交際は里田からのアプローチによるものとのことで、前出の芸能デスクは、「2人は昨年末の番組共演時に、里田から猛アタック。『一緒に食事でも』と誘い、春頃にはつきあい始めた。里田は仲のいいスザンヌ（24）、木下優樹菜（22）には、早くから相談していたし、周囲にも浮かれてしゃべりまくっていた。次世代エースをゲットしたわけですから、賞味期限切れ間近の里田にすれば、してやったりでしょう」とも語っている。

ちなみに、同じく同誌にコメントする球団関係者は、「高校時代は同じ高校の女子生徒と在学中の3年間、ずっとつきあっていましたね。基本はオクテ。プロ入り後にキャバクラなども覚えましたが、まぁ、モテるタイプではないし、女あしらいも不慣れ。高校時代はかなりワガママで、元カノは苦労が絶えなかったと聞いています。同年代の女より、里田のような姉さんタイプのほうが勝負の世界では向いているケースも多いですからね」と明かしており、この交際も田中にとっては“吉”とした。

・週刊アサヒ芸能 ［ライト版］＜デジタル＞（PC版）
・週刊アサヒ芸能（モバイル版）

最もcos類似度が高い文章　documents[5666]
類似度=0.33731426517908353
http://news.livedoor.com/article/detail/5225245/
2010-12-22T10:30:00+0

## なぜそのような結果になったのか考察する


### Cos類似度の出力結果について
#### 比較元に記載されている単語の一部が、出力された文章にも記載されている
- 例　マー君、里田、東北楽天ゴールデンイーグルス、週刊アサヒ芸能、関係者

#### 想定される理由
- 動作原理として文章の全単語をベクトル化したデータから、類似した文章を選択している為、同様の単語を多く含む文章が選択されたと思われる。

### Word Mover's Distanceの出力結果について
#### 文章の位置と内容が近い文章が出力されている
- 例　比較元16行目「ちなみに、同じく同誌にコメントする球団関係者は、」出力14行目「同じく同誌に寄せる芸能関係者は、」

#### 想定される理由
- 動作原理として文書Aを文書Bへ変換する際のコストが最小となる組み合わせを選択している為、同じ位置に同じような文章を含む文章が最も変換ロスが小さい文章と推定されていると思われる。

# 【問題6】感情分析
目的

- NLP定番の感情分析の経験
- 英語の処理の実践
以下からLarge Movie Review Datasetをダウンロードしてください。

[IMDBレビュー](http://ai.stanford.edu/~amaas/data/sentiment/)

同じようにwgetコマンドでも可能です。

```python
# IMDBをカレントフォルダにダウンロード
!wget 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
# 解凍
!tar 'zxf aclImdb_v1.tar.gz'
# aclImdb/train/unsupはラベル無しのため削除
!rm -rf 'aclImdb/train/unsup'
# IMDBデータセットの説明を表示
!cat 'aclImdb/README'
```

```python
# サブフォルダまで自動で読み込んでもらう
from sklearn.datasets import load_files

train_review = load_files('./aclImdb/train/', encoding='utf-8')
train_text, train_y = train_review.data, train_review.target

test_review = load_files('./aclImdb/test/', encoding='utf-8')
test_text, test_y = test_review.data, test_review.target
```

## 【問】
IMDBという映画に対するレビューのデータセットを使います。
良いレビューか悪いレビューかを判定するモデルを作ってください。
テストデータに対する正解率が90%を超えるまで、調査=>実行=>改善を繰り返してください。
前処理になぜその処理をしたのかを書くとエンジニアリングとしても完璧です。
注意: 必ず間違っていたデータを観察してください。

## データの読み込み

### 文章が記載されたデータ
- 作成したモデルで正確に分類出来なかった文章を確認する際に使用

In [131]:
test_review = load_files('./aclImdb/test/', encoding='utf-8')
test_text, test_y = test_review.data, test_review.target

In [132]:
test_text[0]

"Don't hate Heather Graham because she's beautiful, hate her because she's fun to watch in this movie. Like the hip clothing and funky surroundings, the actors in this flick work well together. Casey Affleck is hysterical and Heather Graham literally lights up the screen. The minor characters - Goran Visnjic {sigh} and Patricia Velazquez are as TALENTED as they are gorgeous. Congratulations Miramax & Director Lisa Krueger!"

### 数値化されたデータ
- 学習に使用する
- IMDBデータセットをダウンロードし、学習データを配列trainに、テストデータを配列testへ格納
- 各配列[0]がレビューの単語データ、各配列[1]がレビューの感情ラベル
- 単語データはインデックス化されている

In [107]:
train, test, _ = imdb.load_data(path='./data/imdb.pkl', n_words=10000, valid_portion=0.1)

## 前処理
- 単語データと感情ラベルを別々の配列に分ける（XとY）

In [108]:
# 学習用とテスト用のデータをそれぞれ変数に格納
X_train, y_train = train
X_test, y_test = test

- 単語データの配列Xはサイズを合わせる（ゼロパディング）。

In [109]:
# 学習用とテスト用の単語データのサイズを合わせる
X_train = pad_sequences(X_train, maxlen=100, value=0.)
X_test = pad_sequences(X_test, maxlen=100, value=0.)

- 感情ラベルYは0（肯定）か1（否定）のどちらか2値を持つ、このデータをone-hot-encodingする

In [110]:
# 1つ目のレビュー単語データの中身を確認
X_train[0], X_test[0]

(array([ 17,  25,  10, 406,  26,  14,  56,  61,  62, 323,   4,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([  70,  922,   87,  394,   25,    3,  129,  883,   53,  457,   86,
         879,   87,   70,  297,   42,   41,   86, 1752,   14,   40,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

## OneHotEncoding

In [111]:
# 正解データのサイズを合わせる
y_train = to_categorical(y_train, nb_classes=2)
y_test = to_categorical(y_test, nb_classes=2)

In [112]:
# 1つ目のレビュー正解データの中身を確認
y_train[0], y_test[0]

(array([1., 0.]), array([1., 0.]))

## ニューラルネットワークの作成
- 入力層、中間層（埋め込み層、LSTMブロック）、出力層を作成し、学習条件を設定
- 最適化手法としてAdam Optimizerを使用して重みを更新
- 埋め込み層では、単語の特徴を数値で表現したものが出力される

In [113]:
# 入力層の作成
net = tflearn.input_data([None, 100])

In [114]:
# 中間層の作成
# 単語埋め込み層の作成
net = tflearn.embedding(net, input_dim=10000, output_dim=128)

In [115]:
# LSTMブロック
net = tflearn.lstm(net, 128, dropout=0.8)

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [116]:
# 出力層の作成
net = tflearn.fully_connected(net, 2, activation='softmax')

In [117]:
# 学習条件のセット
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


## モデルの作成
- 作成したニューラルネットワークに対しデータをセットし学習を実行。

In [118]:
# モデルのセット
model = tflearn.DNN(net, tensorboard_verbose=0)

Instructions for updating:
Use tf.cast instead.


## 検証結果　Test Accuracy 0.8228

In [119]:
# 学習の実行
model.fit(X_train, y_train, validation_set=(X_test, y_test), show_metric=True, batch_size=32, run_id='lstm_model')

Training Step: 7039  | total loss: 0.06559 | time: 99.775s
| Adam | epoch: 010 | loss: 0.06559 - acc: 0.9903 -- iter: 22496/22500
Training Step: 7040  | total loss: 0.08002 | time: 103.388s
| Adam | epoch: 010 | loss: 0.08002 - acc: 0.9881 | val_loss: 0.64915 - val_acc: 0.8228 -- iter: 22500/22500
--


## 間違っている文章を確認

In [147]:
y_pred = np.array(model.predict(X_test))
y_pred

array([[0.9907098 , 0.00929019],
       [0.9799039 , 0.02009615],
       [0.01865811, 0.98134196],
       ...,
       [0.06731224, 0.9326878 ],
       [0.6471061 , 0.3528939 ],
       [0.00197308, 0.9980269 ]], dtype=float32)

In [148]:
y_pred_index = y_pred.argmax(axis=1)
y_pred_index

array([0, 0, 1, ..., 1, 0, 1], dtype=int64)

In [124]:
y_test

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [129]:
y_test = y_test.argmax(axis=1)
y_test

array([0, 0, 1, ..., 0, 1, 1], dtype=int64)

## 間違えた文章のindex

In [150]:
np.where(y_pred_index != y_test)

(array([   5,   18,   26,   40,   46,   57,   58,   63,   66,   67,   69,
          89,   92,  136,  141,  150,  154,  159,  163,  166,  170,  178,
         194,  205,  206,  207,  212,  225,  241,  260,  265,  269,  281,
         282,  285,  290,  292,  300,  304,  331,  337,  338,  342,  344,
         346,  354,  355,  358,  360,  372,  373,  381,  382,  385,  395,
         403,  406,  410,  415,  418,  440,  451,  458,  461,  492,  494,
         499,  500,  507,  526,  529,  554,  562,  585,  591,  608,  614,
         615,  626,  629,  630,  633,  638,  649,  657,  664,  670,  674,
         676,  678,  681,  683,  686,  700,  705,  708,  713,  714,  724,
         727,  737,  741,  751,  758,  764,  769,  772,  794,  798,  800,
         801,  804,  810,  811,  816,  817,  853,  856,  866,  884,  895,
         897,  899,  905,  907,  909,  923,  925,  927,  939,  943,  958,
         960,  966,  979,  980,  984,  986,  987,  995, 1003, 1004, 1007,
        1010, 1015, 1016, 1021, 1024, 

## index 5 
- 正解→0(肯定)　予測→1(否定)

In [141]:
y_test[5]

0

In [151]:
y_pred_index[5]

1

In [152]:
y_pred[5]

array([0.11030001, 0.88970006], dtype=float32)

In [133]:
test_text[5]

'"Are You in the House Alone?" belongs to the pre-cable TV days when the networks were eager to offer an alternative to popular TV shows. It is well-made thriller with a talented cast and credible situations. Kathleen Beller plays a High School student who gets a series of threatening letters. Everyone seems to think that it is nothing more than a prank but Beller is really scared. Tony Bill and Blythe Danner play Beller\'s parents, Ellen Travolta (John\'s sister) is the High School Principal and Dennis Quaid has one of his earliest roles as a cocky rich kid. It\'s a competent chiller with a still relevant social message. Beller is lovely - if you are 30 or older, you will remember that she was very popular among youngsters. Blythe Danner, who I usually don\'t like, gives a truly moving performance. Nice little film.'

### 和訳
「あなたは一人で家にいますか？」は、ネットワークが人気のあるテレビ番組に代わるものを提供することを熱望していたケーブルテレビ前の時代に属しています。 Tony BillとBlythe DannerがBellerの両親を演じるのは、Ellen Travolta（Johnの姉妹）である。 High School PrincipalとDennis Quaidは、元気いっぱいの金持ちの子供としての彼の初期の役割の1つを持っていますまだ関連性のあるソーシャルメッセージを持つ有能なチラーですBeller is lovely  - あなたは30歳以上の場合、彼女は非常に人気があったことを覚えている 私は普段好きではないBlythe Dannerが本当に感動的なパフォーマンスを提供します。

### 考察
- 肯定的な文章（「本当に感動的なパフォーマンスを提供します」等）は文章の最後の方しか記載されていない為、否定的と判断したと思われる。

## index 18
- 正解→1(否定)　予測→0(肯定)

In [143]:
y_test[18]

1

In [153]:
y_pred_index[18]

0

In [154]:
y_pred[18]

array([0.9397095 , 0.06029051], dtype=float32)

In [134]:
test_text[18]

"A huge cast gathered for this remake which sadly was a box office failure notwithstanding a great sound track. I can't say it was riveting entertainment, nor a cure for insomnia. Nevertheless I enjoyed the film - it provided the escape I was after one afternoon. A good look for those of us looking for the ideal life, albeit a fantasy. Expect some corny moments, few thrills, and an occasional laugh."

### 和訳
このリメイクのために巨大なキャストが集まりました。悲しいことに、素晴らしいサウンドトラックにもかかわらず興行失敗でした。 私はそれが娯楽を惹きつけていた、または不眠症の治療法であるとは言えません。 それにもかかわらず、私はその映画を楽しんだ - それは私が午後の後にあった逃避を提供した。 ファンタジーにもかかわらず、理想的な生活を探している私たちの人々のための良い外観。 いくつかの愚かな瞬間、いくつかのスリル、そして時折笑いを期待

### 考察
- 否定的な単語より、肯定的な単語(「惹きつけていた」「楽しんだ」等)が多くある為、肯定と判断したと思われる

## index 2498
- 正解→1(否定)　予測→0(肯定)

In [145]:
y_test[2498]

1

In [155]:
y_pred_index[2498]

0

In [156]:
y_pred[2498]

array([0.6471061, 0.3528939], dtype=float32)

In [135]:
test_text[2498]

'I could have done with the seven gunslingers just staying away. This sequel should never have been done, the first did it all and better. The plot was a turkey, the acting was turkey, the direction, production, camera work... all turkey. Whoever put out this junk should be tarred and feathered. May they not return again!'

### 和訳
私は7人の砲手たちがただ遠ざかっているだけで終わったかもしれません。 この続編は行われたことがないはずです、最初はそれがすべてよりよくしました。 プロットは七面鳥、演技は七面鳥、演出、演出、カメラワーク…すべて七面鳥だった。 このゴミを出した人は誰でもうんざりして羽毛が付いているはずです。 二度と戻ってこないように！

### 考察
- 感情的な文章（「二度と戻ってこないように！」等）が多く、各クラスの予測確率が近い値となっている事から、一般的な文章より判断が難しいと思われる。

# 【問題8】自然言語処理の応用事例
## 目的
- NLPの情報共有
- 英語の処理の実践

現在自然言語処理はどのような企業でどのように活用されているか？ 
1つ例をあげて3~5分で発表してください。 
(例)メルカリは商品説明をTF-IDFを用いてベクトル化して商品の異常検知を行っている。

## 調査内容「自然言語処理を駆使しAIが営業活動を分析(ストックマーク株式会社 Asales)」
### ①概要
- AIが「社内の定性データ(資料、メモ等)データを理解」「顧客・業界動向の収集」を行い、営業活動を分析する

### ②何ができるのか？
#### 受失注分析
- 案件のメモや報告書・メールなどのテキストデータから、受失注の要因を分析。
- 4C等のフレームワークをベースに自社の営業活動における強み・弱みを俯瞰。

#### 日報・対話解析
- 日報や日々のメールを自動解析。
- リアルタイムにデータを解析に優先して営業を行うべき案件を自動抽出・提案。
- 日報の要約も自動生成し、報告業務もサポート。

#### 情報武装
- 訪問予定顧客の最新トピックをPush配信し、業界・業種、ソリューション関連情報も網羅。
- 社員の情報感度を向上。

#### 提案資料作成補助
- 過去の類似案件・受注案件の提案資料を自動解析・提案。誰が・いつ・どんな提案をしているかの具体的な特徴を抽出し、提案資料作成を補助。

### 補足情報
#### 4C分析
- 「買う側の視点」に立ってサービスのメリットを整理し、商品の開発や改善のために使われている手法

| 4Cの意味                                      |
|-----------------------------------------------|
| 顧客価値（Customer Value）                  |
| 顧客にとっての経費（Cost）                  |
| 顧客利便性（Convenience）                   |
| 顧客とのコミュニケーション（Communication） |

### 参考リンク
[Asalesについて](https://stockmark.ai/asales/)

[4C分析について](https://hansokunodaigaku.com/keiei_post/3219/)
